# Information Retrieval Baseline Code

## 대회 설명
Information Retrieval 경진대회에 오신 여러분 환영합니다! 🎉 <br>
본 대회에서는 과학 상식에 대한 멀터턴 질문에 대하여 RAG 기술을 활용하여 질문에 적합한 레퍼런스를 추출하고 이를 토대로 답변을 생성하는 어플리케이션을 구현해 봅니다. <br>
LLM은 GPT-3.5 버전을 이용하고, 레퍼런스 추출을 위한 검색엔진 구축은 Elasticsearch를 활용합니다.

## 데이터셋 준비
* 질문에 대한 레퍼런스로 활용될 수 있는 4272개의 과학 상식을 다루는 문서
 * Open Ko LLM Leaderboard에 들어가는 Ko-H4 데이터 중 MMLU, ARC 데이터를 기반으로 생성
 * Question과 answer를 paraphrase (question과 answer를 토대로 GPT-4 활용하여 document 생성)
 * 예시)
  ```
  수소, 산소, 질소 가스의 혼합물에서 평균 속도가 가장 빠른 분자는 수소입니다
. 수소 분자는 가장 가볍고 작은 원자로 구성되어 있기 때문에 다른 분자들보다 더 빠르게 움직>입니다. 이러한 이유로 수소 분자는 주어진 온도에서 가장 빠른 평균 속도를 가지고 있습니다. >수소 분자는 화학 반응에서도 활발하게 참여하며, 수소 연료로도 널리 사용됩니다. 따라서 수소 분자는 주어진 온도에서 평균 속도가 가장 빠른 분자입니다.
  ```

* 가상의 질문 220개
 * 과학 상식 질문 또는 일상 대화 메시지를 수동으로 생성
 * 220개 중 20개는 멀티턴 대화 형태의 질문, 20개는 일상 칫챗 대화
  * 예시)
  ```
{"eval_id": 0, "msg": [{"role": "user", "content": "나무의 분류에 대해 조사해 보기 위한 방
법은?"}], "refs": ["c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d"]}
{"eval_id": 1, "msg": [{"role": "user", "content": "각 나라에서의 공교육 지출 현황에 대해 알려줘."}], "refs": ["79c93deb-fe60-4c81-8d51-cb7400a0a156"]}
{"eval_id": 2, "msg": [{"role": "user", "content": "기억 상실증 걸리면 너무 무섭겠다."}, {"role": "assistant", "content": "네 맞습니다."}, {"role": "user", "content": "어떤 원인 때
문에 발생하는지 궁금해."}], "refs": ["25de4ffd-cee4-4f27-907e-fd6b802c6ede"]}
  ```

In [1]:
cd /home/

/home


/home/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## 환경 설정
검색엔진을 위한 Elasitcsearch, 임베딩 생성을 위한 sentence transformers, LLM 사용을 위한 openai client 설치가 필요합니다.

In [3]:
# daemon user로 구동을 하기 위해 소유자 변경
!chown -R daemon:daemon elasticsearch-8.15.2/

# 코랩 노트북 환경에서 서버 구동을 위한 리소스 제한/격리를 위해 아래 명령 수행
!umount /sys/fs/cgroup

!apt install cgroup-tools

# Nori 형태소 분석기 설치
!/content/elasticsearch-8.15.2/bin/elasticsearch-plugin list analysis-nori
#!/content/elasticsearch-8.15.2/bin/elasticsearch-plugin install analysis-nori  # 설치코드


umount: /sys/fs/cgroup: must be superuser to unmount.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
cgroup-tools is already the newest version (0.41-10).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
/bin/bash: /content/elasticsearch-8.15.2/bin/elasticsearch-plugin: No such file or directory


## 검색엔진 준비 - Elasticsearch




In [41]:
import os
import json
import pandas as pd
import re
from elasticsearch import Elasticsearch, helpers
from subprocess import Popen, PIPE, STDOUT
import time


In [4]:

# 엘라스틱서치의 데몬 인스턴스 만들기
es_server = Popen(['elasticsearch-8.15.2/bin/elasticsearch'],
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

# 인스턴스를 로드하는 데 약간의 시간이 걸림
for i in range(30):
    print("\r",i+1,end="")
    time.sleep(1)

 30

In [5]:
# 데몬이 구동되었는지 확인 (세개의 daemon process가 있어야 함)
!ps -ef | grep elasticsearch

daemon    140113  140000  9 15:06 ?        00:00:03 /home/elasticsearch-8.15.2/jdk/bin/java -Xms4m -Xmx64m -XX:+UseSerialGC -Dcli.name=server -Dcli.script=elasticsearch-8.15.2/bin/elasticsearch -Dcli.libs=lib/tools/server-cli -Des.path.home=/home/elasticsearch-8.15.2 -Des.path.conf=/home/elasticsearch-8.15.2/config -Des.distribution.type=tar -cp /home/elasticsearch-8.15.2/lib/*:/home/elasticsearch-8.15.2/lib/cli-launcher/* org.elasticsearch.launcher.CliToolLauncher
daemon    140173  140113 99 15:06 ?        00:00:56 /home/elasticsearch-8.15.2/jdk/bin/java -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -Djava.security.manager=allow -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Dlog4j2.formatMsgNoLookups=true -Djava.locale

In [ ]:
# 데몬 구동후 password 설정 단계 필요
# 명령 실행 후 "Please confirm that you would like to continue"에서 y 입력 필요
# !/content/elasticsearch-8.8.0/bin/elasticsearch-setup-passwords auto -url "https://localhost:9200"

In [7]:
es_username = 'elastic'

# 위 명령 실행 결과의 마지막 부분인 PASSWORD elastic 값으로 교체 필요
es_password = 'MlXsewAP3QwdAscB0drd'

# Elasticsearch client 생성
es = Elasticsearch(['https://localhost:9200'], basic_auth=(es_username, es_password), ca_certs="/home/elasticsearch-8.15.2/config/certs/http_ca.crt")

# Elasticsearch client 정보 확인
print(es.info())

{'name': 'instance-11864', 'cluster_name': 'elasticsearch', 'cluster_uuid': '_CSckQ_hR9e6NoYNQRjJ6Q', 'version': {'number': '8.15.2', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '98adf7bf6bb69b66ab95b761c9e5aadb0bb059a3', 'build_date': '2024-09-19T10:06:03.564235954Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [17]:
from sentence_transformers import SentenceTransformer

# Sentence Transformer 모델 초기화 (한국어 임베딩 생성 가능한 어떤 모델도 가능)
# model_name = "jhgan/ko-sroberta-multitask"  # 점수 낮음 0.7121
# model_name = "snunlp/KR-SBERT-Medium-extended-patent2024-hn"  # 4-1
model_name = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"

model = SentenceTransformer(model_name)
# https://huggingface.co/snunlp/KR-SBERT-V40K-klueNLI-augSTS
# https://github.com/snunlp/KR-SBERT


# SetntenceTransformer를 이용하여 임베딩 생성
def get_embedding(sentences):
    return model.encode(sentences)


# 주어진 문서의 리스트에서 배치 단위로 임베딩 생성
def get_embeddings_in_batches(docs, batch_size=100):
    batch_embeddings = []
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        contents = [doc["content"] for doc in batch]
        embeddings = get_embedding(contents)
        batch_embeddings.extend(embeddings)
        print(f'batch {i}')
    return batch_embeddings


# 새로운 index 생성
def create_es_index(index, settings, mappings):
    # 인덱스가 이미 존재하는지 확인
    if es.indices.exists(index=index):
        # 인덱스가 이미 존재하면 설정을 새로운 것으로 갱신하기 위해 삭제
        es.indices.delete(index=index)
    # 지정된 설정으로 새로운 인덱스 생성
    es.indices.create(index=index, settings=settings, mappings=mappings)


# 지정된 인덱스 삭제
def delete_es_index(index):
    es.indices.delete(index=index)


# Elasticsearch 헬퍼 함수를 사용하여 대량 인덱싱 수행
def bulk_add(index, docs):
    # 대량 인덱싱 작업을 준비
    actions = [
        {
            '_index': index,
            '_source': doc
        }
        for doc in docs
    ]
    return helpers.bulk(es, actions)


# 역색인을 이용한 검색
def sparse_retrieve(query_str, size):
    query = {
        "match": {
            "content": {
                "query": query_str
            }
        }
    }
    return es.search(index="test", query=query, size=size, sort="_score")


# Vector 유사도를 이용한 검색
def dense_retrieve(query_str, size):
    # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기
    query_embedding = get_embedding([query_str])[0]

    # KNN을 사용한 벡터 유사성 검색을 위한 매개변수 설정
    knn = {
        "field": "embeddings",
        "query_vector": query_embedding.tolist(),
        "k": size,
        "num_candidates": 100
    }

    # 지정된 인덱스에서 벡터 유사도 검색 수행
    return es.search(index="test", knn=knn)


In [110]:
settings = {
    "analysis": {  # Elasticsearch에서 텍스트 분석을 위한 설정을 정의
        "analyzer": {  # 텍스트를 분석하기 위한 분석기 정의 (이 경우 'nori' 분석기)
            "nori": {  # 사용자 정의(custom) 'nori' 분석기 설정
                "type": "custom",  # 이 분석기는 'custom' 타입으로 사용자가 설정한 대로 동작
                "tokenizer": "nori_tokenizer",  # 'nori_tokenizer'를 사용하여 텍스트를 토큰화
                "decompound_mode": "mixed",  # 복합어(예: '책상' -> '책', '상')를 처리하는 방법을 설정 ('mixed' 모드)
                # 복합어 처리 모드:
                # - "mixed": 복합어와 그 구성 요소 둘 다 반환 (예: '책상'과 '책', '상'을 모두 반환)
                # - "none": 복합어를 분리하지 않음
                # - "discard": 복합어를 버리고 구성 요소만 반환
                "filter": ["nori_posfilter"]  # 분석기에서 사용할 필터 설정 (nori_posfilter를 적용)
            }
        },
        "filter": {  # 텍스트 분석에 사용될 필터 정의
            "nori_posfilter": {  # 형태소 분석에서 사용할 필터를 정의 (품사 기반 필터)
                "type": "nori_part_of_speech",  # 'nori' 형태소 분석기의 품사 필터 사용
                "stoptags": ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX"]  # 특정 품사(tag)를 필터링하여 제외
                # stoptags 설명:
                # - "E" (어미): 용언의 끝부분, 동사와 형용사의 변형을 필터링
                # - "J" (조사): 주격 조사, 목적격 조사 등을 필터링
                # - "SC" (구분자): 쉼표, 마침표 등 구분자를 필터링
                # - "SE" (줄임표): 생략부호(...) 등을 필터링
                # - "SF" (마침표): 문장의 끝을 나타내는 마침표 등을 필터링
                # - "VCN" (지정사): 지정사 품사 필터링 (예: '이다'와 같은 서술적 용어)
                # - "VCP" (보조 용언): 보조 동사, 예를 들어 '하다', '되다'와 같은 동작을 필터링
                # - "VX" (보조 용언): 보조 용언, 동사와 결합하여 다른 의미를 형성하는 보조 동사 필터링
            }
        }
    }
}

# 색인을 위한 mapping 설정 (역색인 필드, 임베딩 필드 모두 설정)
mappings = {
    "properties": {
        # "content" 필드는 텍스트 데이터를 저장합니다.
        # 이 필드는 "text" 타입이며, 한국어 분석을 위해 nori 분석기를 사용합니다.
        "content": {
            "type": "text",        # Elasticsearch에서 텍스트 필드 타입
            "analyzer": "nori"     # 한국어 분석기 "nori"를 사용하여 텍스트를 처리
        },
        
        # "embeddings" 필드는 고차원 벡터 데이터를 저장하는 필드입니다.
        # 예를 들어, 자연어 처리에서 문장 임베딩(벡터 표현)을 저장할 수 있습니다.
        # 이 필드는 "dense_vector" 타입으로 설정됩니다.
        "embeddings": {
            "type": "dense_vector",  # 벡터를 저장하는 필드 타입
            "dims": 768,             # 벡터의 차원 수. 여기서는 768차원 벡터를 의미함 (BERT 기반 임베딩 예시)
            
            # 벡터 검색을 위해 인덱싱 활성화 (벡터 기반 검색에 필수)
            "index": True,           # 해당 필드를 인덱싱하여 검색 가능하게 설정
            
            # 벡터 간의 유사도를 측정하는 방식으로 "l2_norm"을 사용 (L2 거리 기반)
            "similarity": "l2_norm"  # L2 노름(유클리디안 거리)를 이용한 유사도 계산 방식
        }
    }
}




In [101]:
# settings, mappings 설정된 내용으로 'test' 인덱스 생성
create_es_index("test", settings, mappings)

In [102]:
# 문서의 content 필드에 대한 임베딩 생성
index_docs = []
with open("/home/data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]
embeddings = get_embeddings_in_batches(docs)

# 생성한 임베딩을 색인할 필드로 추가
for doc, embedding in zip(docs, embeddings):
    doc["embeddings"] = embedding.tolist()
    index_docs.append(doc)

# 'test' 인덱스에 대량 문서 추가
ret = bulk_add("test", index_docs)

# 색인이 잘 되었는지 확인 (색인된 총 문서수가 출력되어야 함)
print(ret)


batch 0
batch 100
batch 200
batch 300
batch 400
batch 500
batch 600
batch 700
batch 800
batch 900
batch 1000
batch 1100
batch 1200
batch 1300
batch 1400
batch 1500
batch 1600
batch 1700
batch 1800
batch 1900
batch 2000
batch 2100
batch 2200
batch 2300
batch 2400
batch 2500
batch 2600
batch 2700
batch 2800
batch 2900
batch 3000
batch 3100
batch 3200
batch 3300
batch 3400
batch 3500
batch 3600
batch 3700
batch 3800
batch 3900
batch 4000
batch 4100
batch 4200
(4272, [])


### <font color=orange> 역색인을 사용하는 검색 예제

In [103]:
# 검색엔진에 색인이 잘 되었는지 테스트하기 위한 질의
test_query1 = "평균을 구하는 코드를 작성중이야. 네 무엇을 도와드릴까요? 예외처리가 필요한 경우를 알려줘."
test_query2 = """
1. 예외처리가 필요한 경우에 대해 알고 싶어하는 정보가 무엇인가요?

2. 예외처리는 프로그램이 실행 중에 예기치 않은 상황이 발생했을 때, 이를 적절하게 처리하는 것을 의미합니다. 예를 들어, 사용자가 잘못된 입력을 했을 때, 파일을 찾을 수 없을 때 등의 상황에서 예외처리가 필요합니다.

3. 핵심주제:
   - 예외 상황 : 프로그램 실행 중에 발생할 수 있는 예외적인 상황에 대한 이해
   - 예외처리의 필요성 : 예외 상황을 적절히 처리하여 프로그램의 안정성과 신뢰성을 높이는 방법에 대한 이해
   - 예외처리 방법 : try-catch 블록을 사용하여 예외를 처리하거나, 예외를 던지고 처리하는 방법에 대한 이해
"""
test_query3 = "평균, 코드, 예외처리"

In [104]:
search_result_retrieve = sparse_retrieve(test_query1, 3)
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

score: 21.240307 source: 평균을 찾는 알고리즘은 평균 = 합계 / N이며, 여기서 N과 합계는 모두 정수입니다. 이 알고리즘을 사용하는 프로그램에서 프로그래머가 N이 0인지 확인하는 테스트를 포함하는 것을 잊었습니다. N이 0인 경우, 오류는 실행 시간 동안 감지됩니다. 프로그램이 실행되는 동안, 알고리즘은 N이 0인지 확인하고, 만약 N이 0이라면 오류를 감지하고 적절한 예외 처리를 수행합니다. 이렇게 함으로써 프로그램은 잘못된 입력에 대해 안정적으로 처리할 수 있습니다. 따라서, 프로그램이 실행되는 동안 N이 0인 경우 오류가 감지되고 처리됩니다.
score: 17.165508 source: 버퍼 오버플로우는 경계 검사가 완전히 수행되지 않으면 앱에서 버그로 남을 수 있습니다. 버퍼 오버플로우는 프로그래밍에서 발생하는 보안 취약점 중 하나로, 입력 데이터가 버퍼의 크기를 초과하여 인접한 메모리 영역을 침범하는 상황을 의미합니다. 이러한 상황에서는 악의적인 공격자가 앱의 실행 흐름을 조작하거나 악성 코드를 실행할 수 있습니다. 따라서, 경계 검사는 버퍼 오버플로우를 방지하기 위해 반드시 수행되어야 합니다. 경계 검사는 입력 데이터의 크기를 확인하고, 버퍼의 크기를 초과하는 경우에는 적절한 예외 처리를 수행하여 앱의 안정성을 보장합니다. 앱 개발자들은 경계 검사를 통해 버퍼 오버플로우와 같은 보안 취약점을 예방하고, 사용자의 개인정보와 시스템의 안전을 보호해야 합니다.
score: 17.13562 source: 컴파일러는 실행 가능한 소프트웨어를 개발하는 과정에서 매우 중요한 역할을 수행합니다. 이는 추상적인 고급 언어로 작성된 프로그램을 동일한 동작을 표현하는 기계 코드로 번역하는 역할을 맡고 있습니다. 컴파일러는 프로그래머가 작성한 코드를 분석하고, 문법적인 오류를 검출하며, 최적화 기법을 적용하여 실행 속도를 향상시킵니다. 또한, 컴파일러는 프로그램의 구조를 분석하여 메모리 할당과 같은 중요한 작업을 수행합니다. 이러한 과정을 통해 컴

In [105]:
search_result_retrieve = sparse_retrieve(test_query2, 3)
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

score: 187.14243 source: 평균을 찾는 알고리즘은 평균 = 합계 / N이며, 여기서 N과 합계는 모두 정수입니다. 이 알고리즘을 사용하는 프로그램에서 프로그래머가 N이 0인지 확인하는 테스트를 포함하는 것을 잊었습니다. N이 0인 경우, 오류는 실행 시간 동안 감지됩니다. 프로그램이 실행되는 동안, 알고리즘은 N이 0인지 확인하고, 만약 N이 0이라면 오류를 감지하고 적절한 예외 처리를 수행합니다. 이렇게 함으로써 프로그램은 잘못된 입력에 대해 안정적으로 처리할 수 있습니다. 따라서, 프로그램이 실행되는 동안 N이 0인 경우 오류가 감지되고 처리됩니다.
score: 152.0506 source: 버퍼 오버플로우는 경계 검사가 완전히 수행되지 않으면 앱에서 버그로 남을 수 있습니다. 버퍼 오버플로우는 프로그래밍에서 발생하는 보안 취약점 중 하나로, 입력 데이터가 버퍼의 크기를 초과하여 인접한 메모리 영역을 침범하는 상황을 의미합니다. 이러한 상황에서는 악의적인 공격자가 앱의 실행 흐름을 조작하거나 악성 코드를 실행할 수 있습니다. 따라서, 경계 검사는 버퍼 오버플로우를 방지하기 위해 반드시 수행되어야 합니다. 경계 검사는 입력 데이터의 크기를 확인하고, 버퍼의 크기를 초과하는 경우에는 적절한 예외 처리를 수행하여 앱의 안정성을 보장합니다. 앱 개발자들은 경계 검사를 통해 버퍼 오버플로우와 같은 보안 취약점을 예방하고, 사용자의 개인정보와 시스템의 안전을 보호해야 합니다.
score: 123.109474 source: 정수 오버플로우는 프로그램에서 정수를 저장할 공간이 더 이상 없을 때 발생합니다. 이는 정수형 변수가 표현할 수 있는 범위를 초과하여 값을 저장하려고 할 때 발생합니다. 예를 들어, 32비트 정수형 변수는 -2,147,483,648부터 2,147,483,647까지의 값을 저장할 수 있습니다. 만약 이 범위를 초과하는 값을 저장하려고 하면 정수 오버플로우가 발생합니다. 이는 프로그램의 실행에 심각한 문제를 일으킬 수 있으며, 예기치 

In [106]:
search_result_retrieve = sparse_retrieve(test_query3, 3)
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

score: 17.524582 source: 평균을 찾는 알고리즘은 평균 = 합계 / N이며, 여기서 N과 합계는 모두 정수입니다. 이 알고리즘을 사용하는 프로그램에서 프로그래머가 N이 0인지 확인하는 테스트를 포함하는 것을 잊었습니다. N이 0인 경우, 오류는 실행 시간 동안 감지됩니다. 프로그램이 실행되는 동안, 알고리즘은 N이 0인지 확인하고, 만약 N이 0이라면 오류를 감지하고 적절한 예외 처리를 수행합니다. 이렇게 함으로써 프로그램은 잘못된 입력에 대해 안정적으로 처리할 수 있습니다. 따라서, 프로그램이 실행되는 동안 N이 0인 경우 오류가 감지되고 처리됩니다.
score: 13.512451 source: 버퍼 오버플로우는 경계 검사가 완전히 수행되지 않으면 앱에서 버그로 남을 수 있습니다. 버퍼 오버플로우는 프로그래밍에서 발생하는 보안 취약점 중 하나로, 입력 데이터가 버퍼의 크기를 초과하여 인접한 메모리 영역을 침범하는 상황을 의미합니다. 이러한 상황에서는 악의적인 공격자가 앱의 실행 흐름을 조작하거나 악성 코드를 실행할 수 있습니다. 따라서, 경계 검사는 버퍼 오버플로우를 방지하기 위해 반드시 수행되어야 합니다. 경계 검사는 입력 데이터의 크기를 확인하고, 버퍼의 크기를 초과하는 경우에는 적절한 예외 처리를 수행하여 앱의 안정성을 보장합니다. 앱 개발자들은 경계 검사를 통해 버퍼 오버플로우와 같은 보안 취약점을 예방하고, 사용자의 개인정보와 시스템의 안전을 보호해야 합니다.
score: 10.144779 source: 프로그래머는 아래의 코드 세그먼트를 작성하여 numbers라는 목록에 있는 모든 요소의 평균을 표시했습니다. 목록에는 최소 한 개 이상의 숫자가 항상 있습니다.

 Line 1: count ← 0
 Line 2: sum ← 0
 Line 3: FOR EACH value IN numbers
 Line 4: {
 Line 5: count ← count + 1
 Line 6: sum ← sum + value
 Line 7: average

### <font color=orange> 백터 유사도 검색 예제

In [107]:
search_result_retrieve = dense_retrieve(test_query1, 3)
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

score: 0.72962093 source: 보간 다항식은 주어진 데이터 포인트를 통해 함수를 근사하는 다항식입니다. P(x)가 n + 1개의 서로 다른 실수 x0, .... Xn에서 실수 함수 f(x)에 대한 최소 차수 보간 다항식인 경우, p(x)의 최대 가능 차수는 n입니다. 이는 주어진 n + 1개의 데이터 포인트를 모두 지나는 다항식을 만들기 위해 필요한 최소 차수입니다. 따라서, p(x)의 최대 가능 차수는 n입니다.
score: 0.7211857 source: 헤르미트 연산자의 고유값은 항상 실수입니다. 이는 양자역학에서 중요한 개념으로, 헤르미트 연산자는 에르미트 곱셈법칙을 만족하는 선형 연산자입니다. 이 연산자의 고유값은 실수로 나타납니다. 이는 고유값 문제를 풀 때 중요한 정보로 활용됩니다. 헤르미트 연산자의 고유값이 실수인 이유는 이 연산자가 에르미트 곱셈법칙을 만족하기 때문입니다. 이는 양자역학에서 중요한 성질로, 실제 실험 결과와도 일치하는 결과를 도출할 수 있습니다. 따라서 헤르미트 연산자의 고유값은 항상 실수입니다.
score: 0.71672654 source: 평균을 찾는 알고리즘은 평균 = 합계 / N이며, 여기서 N과 합계는 모두 정수입니다. 이 알고리즘을 사용하는 프로그램에서 프로그래머가 N이 0인지 확인하는 테스트를 포함하는 것을 잊었습니다. N이 0인 경우, 오류는 실행 시간 동안 감지됩니다. 프로그램이 실행되는 동안, 알고리즘은 N이 0인지 확인하고, 만약 N이 0이라면 오류를 감지하고 적절한 예외 처리를 수행합니다. 이렇게 함으로써 프로그램은 잘못된 입력에 대해 안정적으로 처리할 수 있습니다. 따라서, 프로그램이 실행되는 동안 N이 0인 경우 오류가 감지되고 처리됩니다.


In [108]:
search_result_retrieve = dense_retrieve(test_query2, 3)
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

score: 0.8263788 source: 버퍼 오버런은 개발자들이 만든 널리 퍼져 있는 앱의 코딩 오류로써, 공격자가 시스템에 접근하거나 시스템 오작동을 일으키는 데 이용할 수 있습니다. 이 오류는 일반적으로 프로그램이 메모리 버퍼를 초과하여 데이터를 쓰거나 읽을 때 발생합니다. 버퍼 오버런은 보안 취약점으로 간주되며, 악의적인 공격자가 이를 악용하여 시스템을 침해할 수 있습니다. 이러한 오류를 방지하기 위해 개발자들은 코딩 시에 메모리 버퍼의 크기를 철저히 검사하고, 적절한 예외 처리를 수행해야 합니다. 또한, 보안 업데이트와 패치를 정기적으로 적용하여 시스템을 최신 상태로 유지하는 것이 중요합니다. 버퍼 오버런은 앱 개발에서 흔히 발생하는 문제이지만, 적절한 대응과 예방을 통해 시스템의 안전성을 확보할 수 있습니다.
score: 0.81457424 source: 컴파일러가 생성한 객체 모듈에는 때때로 모든 소스 프로그램 이름을 그들의 주소에 매핑하는 정보(심볼 테이블에서)가 포함됩니다. 이 정보의 가장 가능성 있는 목적은 디버깅 도구로의 입력으로 사용하기 위해서입니다. 디버깅 도구는 소스 코드의 오류를 찾고 수정하는 데 도움을 주는 도구입니다. 따라서, 컴파일러가 생성한 객체 모듈에는 소스 프로그램 이름과 주소를 매핑하는 정보가 포함되어 있어야 합니다. 이 정보를 통해 디버깅 도구는 소스 코드의 특정 부분을 식별하고 해당 부분에서 발생하는 오류를 추적할 수 있습니다. 따라서, 심볼 테이블에 포함된 정보는 디버깅 도구의 입력으로 사용되어 소스 코드의 오류를 신속하게 해결할 수 있도록 도와줍니다.
score: 0.8137028 source: 스택에서 버퍼 오버플로우는 공격자가 주입한 코드의 실행을 용이하게 하는 방법 중 하나입니다. 이 공격은 스택 메모리의 버퍼를 넘어서 다른 메모리 영역을 침범하여 악의적인 코드를 실행시킬 수 있습니다. 이를 방지하기 위해서는 해당 코드의 위치를 가리키도록 반환 주소를 덮어씌우는 방법을 사용할 수 있습니다. 이렇게 하면 공

In [109]:
search_result_retrieve = dense_retrieve(test_query3, 3)
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

score: 0.73560476 source: 보간 다항식은 주어진 데이터 포인트를 통해 함수를 근사하는 다항식입니다. P(x)가 n + 1개의 서로 다른 실수 x0, .... Xn에서 실수 함수 f(x)에 대한 최소 차수 보간 다항식인 경우, p(x)의 최대 가능 차수는 n입니다. 이는 주어진 n + 1개의 데이터 포인트를 모두 지나는 다항식을 만들기 위해 필요한 최소 차수입니다. 따라서, p(x)의 최대 가능 차수는 n입니다.
score: 0.7198422 source: 헤르미트 연산자의 고유값은 항상 실수입니다. 이는 양자역학에서 중요한 개념으로, 헤르미트 연산자는 에르미트 곱셈법칙을 만족하는 선형 연산자입니다. 이 연산자의 고유값은 실수로 나타납니다. 이는 고유값 문제를 풀 때 중요한 정보로 활용됩니다. 헤르미트 연산자의 고유값이 실수인 이유는 이 연산자가 에르미트 곱셈법칙을 만족하기 때문입니다. 이는 양자역학에서 중요한 성질로, 실제 실험 결과와도 일치하는 결과를 도출할 수 있습니다. 따라서 헤르미트 연산자의 고유값은 항상 실수입니다.
score: 0.7156484 source: 파이썬 3에서 람다 함수는 간단한 익명 함수를 만들 수 있는 기능입니다. 예를 들어, r = lambda q: q * 2라고 할 때, r(3)은 6이 됩니다. 이는 입력값인 3을 2배로 곱한 결과를 반환하는 함수입니다. 람다 함수는 한 줄로 간결하게 작성할 수 있어서 코드의 가독성을 높일 수 있습니다. 파이썬에서는 람다 함수를 자주 사용하여 간단한 연산이나 필터링 작업을 수행할 수 있습니다.


## RAG 구현

준비된 검색엔진과 LLM을 활용하셔 대화형 RAG 구현

In [12]:
from openai import OpenAI
import traceback

# OpenAI API 키를 환경변수에 설정
os.environ["OPENAI_API_KEY"] = "sk-jmGgRmhYloyOgCy6xvQTIxyPQWjzK1PJKVaH_nXzhzT3BlbkFJ94gpbAuXM_lO63FxLn6pJSTIiJN9_6zPr8eDtQGdwA"
#os.environ["OPENAI_API_KEY"] = "Your API Key"

client = OpenAI()
# 사용할 모델을 설정(여기서는 gpt-3.5-turbo-1106 모델 사용)
llm_model = "gpt-3.5-turbo-1106"


In [64]:
# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
persona_function_calling = """
## 역할: 질문 분류 전문가

## 지침
- 사용자의 대화 메시지에 대해서 분석, 분류하고 아래 1, 2 중 해당되는 것 하나를 실행한다.
1. 개인적인 감정 표현, 상호작용 및 자기소개 요청, 대화 종료 요청 또는 일상적인 인사, 정서적 상태 변화에 대한 대화 메시지에는 적절한 대답을 생성한다.
2. 정보 탐구 또는 구체적인 질문, 특정 주제에 대한 공부 및 탐구, 이야기나 사실 요청의 대화 메시지에는 search api를 호출한다.
"""

# GPT 내용 "개인적인 감정, 일상 대화 또는 자아에 관한 질문"과 "정보 요청 또는 특정 주제에 대한 지식 탐구"로 나눌 수 있습니다.


persona_qa = """
## 역할: 다학제적 전문가 또는 다분야 지식전문가

## 지침
- 사용자의 메시지 분석하고 알고 싶어하는 정보가 무엇인지 명확하게하고, 분석된 핵심 주제를 3가지로 정해 관련 순서대로 핵심 주제의 뜻을 확인한다.
- 핵심주제 뜻과 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 한국어로 답변을 생성한다.
"""


# Function calling에 사용할 함수 정의
tools = [
    {
        "type": "function",
        "function": {
            #"name": "search",
            #"description": "search relevant documents",
            "name": "search",  # 'search' -> '검색'으로 변경
            "description": "관련 문서를 검색합니다",  # 한글 설명으로 변경            
            "parameters": {
                "properties": {
                    "standalone_query": {
                        "type": "string",
                        #"description": "Final query suitable for use in search from the user messages history."
                        "description": "사용자의 대화 내역에서 검색에 적합한 최종 쿼리입니다."  # 한글 설명으로 변경
                    }
                },
                "required": ["standalone_query"],
                "type": "object"
            }
        }
    },
]


In [65]:
# LLM과 검색엔진을 활용한 RAG 구현
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
    msg = [{"role": "system", "content": persona_function_calling}] + messages
    try:
        result = client.chat.completions.create(
            model=llm_model,
            messages=msg,
            tools=tools,
            #tool_choice={"type": "function", "function": {"name": "search"}},
            temperature=0,
            seed=1,
            timeout=10
        )
    except Exception as e:
        traceback.print_exc()
        return response

    # 검색이 필요한 경우 검색 호출후 결과를 활용하여 답변 생성
    if result.choices[0].message.tool_calls:
        tool_call = result.choices[0].message.tool_calls[0]
        function_args = json.loads(tool_call.function.arguments)
        standalone_query = function_args.get("standalone_query")

        # Baseline으로는 sparse_retrieve만 사용하여 검색 결과 추출
        search_result = sparse_retrieve(standalone_query, 3)

        response["standalone_query"] = standalone_query
        retrieved_context = []
        for i,rst in enumerate(search_result['hits']['hits']):
            retrieved_context.append(rst["_source"]["content"])
            response["topk"].append(rst["_source"]["docid"])
            response["references"].append({"score": rst["_score"], "content": rst["_source"]["content"]})

        content = json.dumps(retrieved_context)
        messages.append({"role": "assistant", "content": content})
        msg = [{"role": "system", "content": persona_qa}] + messages
        try:
            qaresult = client.chat.completions.create(
                    model=llm_model,
                    messages=msg,
                    temperature=0,
                    seed=1,
                    timeout=30
                )
        except Exception as e:
            traceback.print_exc()
            return response
        response["answer"] = qaresult.choices[0].message.content

    # 검색이 필요하지 않은 경우 바로 답변 생성
    else:
        response["answer"] = result.choices[0].message.content

    return response



In [66]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            if idx > 3:
                break
            j = json.loads(line)
            print(f'{idx}  Question: {j["msg"]}')
            
            response = answer_question(j["msg"])
            print(f'  Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1
    


In [69]:
data_answers = pd.read_json("/home/02-1.csv", lines=True, encoding="utf-8")
data_answers

# 파일을 텍스트로 읽어서 첫 몇 줄 확인
# with open("/home/data/01-1 질문개선01.jsonl", "r", encoding="utf-8") as file:
#     for _ in range(5):  # 처음 5줄만 확인
#         print(file.readline())

#display(data_answers['msg'][0])

#data = {"eval_id": 78, "standalone_query": "나무의 분류 방법", "topk": ["c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d", "c18592d1-12b6-4d45-9fcd-efbdd6d7b016", "8018337f-15cb-4341-b6fa-e311b4372df9"], "answer": "나무의 분류를 조사하기 위한 방법은 다양합니다. 일반적으로 나무의 잎 모양, 줄기 모양, 꽃과 열매의 유무 등을 관찰하여 분류합니다. 또한 나무의 특징을 토대로 한국의 토착 식물에 대한 정보를 참고하여 분류할 수도 있습니다. 분류는 다양한 방법으로 이루어지며, 각각의 방법은 특정한 목적에 따라 유용합니다. 따라서 나무의 분류를 위해서는 다양한 정보와 방법을 활용하여야 합니다.", "references": [{"score": 13.007582, "content": "한 학생이 다양한 종류의 나무를 조사하고 있습니다. 이 학생은 성장 속도, 온도 범위, 크기가 비슷한 두 나무를 발견했습니다. 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다. 이러한 특징을 고려하면, 이 나무들은 대체로 같은 속에 속해 있을 것으로 추측됩니다. 같은 속에 속한 나무들은 종류별로 유사한 특징을 가지고 있으며, 이는 생물 분류학에서 중요한 기준 중 하나입니다. 따라서 이 학생의 조사 결과는 나무의 분류와 관련된 중요한 정보를 제공할 수 있습니다. 이러한 조사는 나무의 성장과 생태에 대한 이해를 높이는 데 도움이 될 것입니다."}, {"score": 11.53242, "content": "별은 많은 다른 방법으로 분류할 수 있습니다. 이 중에서 별을 분류하는 데 가장 유용하지 않은 것은 표면 질감입니다. 별은 종종 밤하늘에서의 겉보기 밝기로 분류됩니다. 그러나 표면 질감은 별을 분류하는 데에는 유용하지 않습니다. 표면 질감은 별의 물리적 특성과는 관련이 없으며, 별의 분류에는 다른 요소들이 더 중요합니다. 예를 들어, 별의 크기, 온도, 스펙트럼 등이 별을 분류하는 데에 더 유용한 정보를 제공합니다. 따라서 표면 질감은 별을 분류하는 데에는 가장 유용하지 않은 방법입니다."}, {"score": 11.24757, "content": "나무는 재생 가능한 에너지원으로 간주됩니다. 이는 나무가 석탄이 형성되는 것보다 빠르게 자라기 때문입니다. 나무는 태양 에너지를 흡수하여 광합성을 통해 자라납니다. 이 과정에서 나무는 이산화탄소를 흡수하고 산소를 방출합니다. 이러한 특성으로 인해 나무는 지속적으로 재생되는 에너지원으로 사용될 수 있습니다. 반면, 석탄은 수백만 년 동안 지속적인 압력과 열에 노출되어 형성됩니다. 석탄은 화석 연료로 분류되며, 한 번 사용되면 재생되지 않습니다. 따라서 나무는 석탄보다 더욱 지속 가능한 에너지원으로 간주됩니다."}]}



# 각 줄에 포함된 리스트나 딕셔너리를 개별 열로 분리
data_refer = pd.json_normalize(data_answers['references'])

display(data_refer)
tmp = str(data_refer.loc[0,0]) + '\n' + str(data_refer.loc[0,1]) + '\n' + str(data_refer.loc[0,2])
print(data_refer.loc[0,0]['score'])
#type(data_refer.loc[0,0])
print(tmp)

,0,1,2
0,"{'score': 20.857122, 'content': '한 학생이 다양한 종류의...","{'score': 16.680271, 'content': '생물학에서 일부 생물체의...","{'score': 14.2015295, 'content': '명왕성은 1930년에 ..."
1,"{'score': 17.829744, 'content': '2017년 현재, 전세계...","{'score': 12.250196, 'content': '전기력은 전하 사이에 작...","{'score': 11.773132, 'content': '세계 의료비 총지출은 2..."
2,"{'score': 18.171331, 'content': '기억 상실은 대뇌의 기능...","{'score': 16.948849, 'content': '노인들이 젊은 성인들보다...","{'score': 12.981144, 'content': '80세 이상의 노인들은 ..."
3,"{'score': 11.356079, 'content': '학교 버스는 동일한 거리...","{'score': 11.186901, 'content': '한 대의 버스가 두 도시...","{'score': 11.173609, 'content': '인텔 8085A 마이크로..."


20.857122
{'score': 20.857122, 'content': '한 학생이 다양한 종류의 나무를 조사하고 있습니다. 이 학생은 성장 속도, 온도 범위, 크기가 비슷한 두 나무를 발견했습니다. 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다. 이러한 특징을 고려하면, 이 나무들은 대체로 같은 속에 속해 있을 것으로 추측됩니다. 같은 속에 속한 나무들은 종류별로 유사한 특징을 가지고 있으며, 이는 생물 분류학에서 중요한 기준 중 하나입니다. 따라서 이 학생의 조사 결과는 나무의 분류와 관련된 중요한 정보를 제공할 수 있습니다. 이러한 조사는 나무의 성장과 생태에 대한 이해를 높이는 데 도움이 될 것입니다.'}
{'score': 16.680271, 'content': '생물학에서 일부 생물체의 분류 방법이 변경되었습니다. 이제 생물체를 재분류하는 데에는 구조보다는 분자 수준에서의 조사가 사용됩니다. 이 새로운 방법은 생물체의 유전자나 단백질의 구조와 기능을 분석하여 그들의 진화적 관계를 밝히는 데에 큰 도움이 됩니다. 이러한 분자 수준의 조사는 생물체의 유전적 유사성을 파악하고, 서로 다른 종 간의 진화적 연결고리를 찾는 데에 중요한 역할을 합니다. 이 방법은 생물체의 분류를 더욱 정확하고 명확하게 만들어주며, 생물 다양성 연구에도 큰 기여를 하고 있습니다. 이제 구조보다는 분자 수준에서의 조사가 생물체의 재분류에 사용되고 있으며, 이는 생물학의 발전에 새로운 지평을 열어주고 있습니다.'}
{'score': 14.2015295, 'content': "명왕성은 1930년에 발견되어 9번째 행성으로 명명되었습니다. 그러나 2003년에는 천문학자 마이클 브라운이 명왕성보다 큰 천체를 발견했습니다. 이로 인해 명왕성은 '왜행성'으로 재분류되었습니다. 이것은 과학적 조사의 결과로서, 새로운 증거가 수집됨에 따라 분류가 변경될 수 있다는 것을 시사합니다. 과학은 지속적인 연구와 발전을 통해 우리의 이해를 확장시키고, 이전에 받아들여진 사실들을 수정하거나 보완할 수 

In [68]:
%%time
# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
input_file  = "/home/data/eval.jsonl"
output_file = "/home/02-1.csv"

msgs = "\"role\": \"system\", \"content:\"\n"
print("LLM_model  : ", llm_model, "\nSentenceTransformer_name : ", model_name, "\n\n")
print(msgs, persona_function_calling)

eval_rag(input_file, output_file)

LLM_model  :  gpt-3.5-turbo-1106 
SentenceTransformer_name :  snunlp/KR-SBERT-V40K-klueNLI-augSTS 


"role": "system", "content:"
 
## 역할: 질문 분류 전문가

## 지침
- 사용자의 대화 메시지에 대해서 분석, 분류하고 아래 1, 2 중 해당되는 것 하나를 실행한다.
1. 개인적인 감정 표현, 상호작용 및 자기소개 요청, 대화 종료 요청 또는 일상적인 인사, 정서적 상태 변화에 대한 대화 메시지에는 적절한 대답을 생성한다.
2. 정보 탐구 또는 구체적인 질문, 특정 주제에 대한 공부 및 탐구, 이야기나 사실 요청의 대화 메시지에는 search api를 호출한다.

0  Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]
  Answer: 나무의 분류를 조사하기 위한 방법은 다양합니다. 가장 먼저는 나무의 잎 모양, 줄기 모양, 꽃과 열매의 유무 등을 관찰하여 나무를 분류할 수 있습니다. 이러한 특징을 파악하면, 해당하는 나무들은 특정한 분류범주로 묶일 수 있습니다. 이러한 분류는 나무의 생태학과 생물학적 특성을 이해하는 데 도움이 됩니다. 또한, 나무의 분류를 위해서는 식물학적인 지식과 분류학적인 방법을 사용해야 합니다. 이러한 방법은 나무의 생태학적 특성을 파악하고, 다양한 환경에서의 적응력을 이해하는 데 도움이 됩니다. 나무의 분류는 1930년대에 처음으로 제안되어 9번째 수정된 버전으로 현재까지 발전해왔습니다. 이러한 분류는 생물학적인 특성과 환경적인 측면을 고려하여 이루어지며, 우리의 이해를 돕고 환경보호에 기여합니다.

1  Question: [{'role': 'user', 'content': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}]
  Answer: 1. 공교육 지출 현황
   - 2017년 전 세계의 공교육 지출은 해당 국가의 GDP의 평균 4%를 占하고 

In [20]:
!wc -l sample_submission.csv

6 sample_submission.csv


#Reference

## Required Package

openai==1.7.2 <br>
elasticsearch==8.8.0 <br>
sentence_transformers==2.2.2 <br>



## 콘텐츠 라이선스

저작권 : <font color='blue'> <b> ©2023 by Upstage X fastcampus Co., Ltd. All rights reserved.</font></b>

<font color='red'><b>WARNING</font> : 본 교육 콘텐츠의 지식재산권은 업스테이지 및 패스트캠퍼스에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다. </b>